### 01. High Price Predict Model with Linear Regression

In [1]:
# 시간 측정 시작
import time
start = time.time()

# 라이브러리
import pandas as pd
kospi = pd.read_csv('./KOSPI.csv')

# KOSPI에 상장된 종목 코드
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))
code_list = kospi['종목코드'].unique()

# 라이브러리
import numpy as np

# 진행과정 Check
check = 0

# 결과 저장 데이터프레임
result_df = pd.DataFrame()

# 코드 별 진행
for code in code_list[:] : 

    # Hyper Parameter
    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   # 학습 기간 시작점
    end_date = '20200229'     # 학습 기간 종료점
    Lookup = '9999' 
    
    # url
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    # url open
    from urllib.request import urlopen
    url_page = urlopen(url)

    # json 파일로 받아오기
    import json
    url_data = json.loads(url_page.read())

    # json을 데이터프레임으로 바꾸기
    import pandas as pd
    data = pd.DataFrame(url_data)
    
    # Data Column명 변경
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
    # 순서 뒤집기
    data = data[::-1]
    data = data.set_index('date')
    
    # 필요한 변수만 추출
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    # 만약에 데이터가 비어있다면 그 종목은 PASS
    if data["open"][0] == '' : 
        continue
    
    # 변수 속성 정수로 바꾸기
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
    # 거래량과 거래가격을 모두 로그 변환
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    # 각각의 피쳐들에 대해 
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    
    # 타겟 생성 : 다음날의 고가를 예측
    data['y'] = data['high'].shift(-2)
    
    # NaN이 들어가 있는 행 제거
    data = data.dropna()
    
    # Input과 Target을 분리
    X = data[data.columns[:-1]]
    y = data['y']
    
    # Linear Regression
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    # Test Data 생성하기
    start_date, end_date = '20200301', '20200318'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    url_page = urlopen(url)
    url_data = json.loads(url_page.read())
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-2)
        
    data = data[-1:]
    
    data = data[data.columns[:-1]]

    # 필요한것들
    result = dict({'종목' : code,
                   '어제 날짜' : data.index.values[0],
                   '어제 종가' : int(data['close'].values),
                   '오늘 고가' : int(round(reg.predict(data)[0])),
                   '오늘 고가 변동폭 예측' : 100*(int(round(reg.predict(data)[0]))-int(data['close'].values))/\
                                     int(data['close'].values)})
    result_df = result_df.append(result, ignore_index=True)
    
    # 각 종목별 결과 출력
    if check % 100 == 0 : 
        print('{:4} / {:4} 종목 ====> {:7.2f}% 완료  {:.4f}'.format(check+1,2144,100*check/2144,time.time()-start))
    check += 1

# 전체 완료
print('{:4} / {:4} 종목 ====> {:7.2f}% 완료  {:.4f}'.format(check + 1,2144,100*check/2144,time.time()-start))

# 시간측정 완료
print("\n\nall time :", time.time() - start)

   1 / 2144 종목 ====>    0.00% 완료  1.7702
 101 / 2144 종목 ====>    4.66% 완료  66.3070
 201 / 2144 종목 ====>    9.33% 완료  130.6555
 301 / 2144 종목 ====>   13.99% 완료  198.3534
 401 / 2144 종목 ====>   18.66% 완료  265.5259
 501 / 2144 종목 ====>   23.32% 완료  331.4518
 601 / 2144 종목 ====>   27.99% 완료  394.3874
 701 / 2144 종목 ====>   32.65% 완료  462.1041
 801 / 2144 종목 ====>   37.31% 완료  527.6558
 901 / 2144 종목 ====>   41.98% 완료  593.2444
1001 / 2144 종목 ====>   46.64% 완료  659.9047
1101 / 2144 종목 ====>   51.31% 완료  739.0072
1201 / 2144 종목 ====>   55.97% 완료  813.9904
1301 / 2144 종목 ====>   60.63% 완료  880.6559
1401 / 2144 종목 ====>   65.30% 완료  947.6039
1501 / 2144 종목 ====>   69.96% 완료  1017.0324
1601 / 2144 종목 ====>   74.63% 완료  1083.7903
1701 / 2144 종목 ====>   79.29% 완료  1151.6733
1801 / 2144 종목 ====>   83.96% 완료  1225.6591
1901 / 2144 종목 ====>   88.62% 완료  1325.1855
2001 / 2144 종목 ====>   93.28% 완료  1392.5871
2101 / 2144 종목 ====>   97.95% 완료  1460.3870
2145 / 2144 종목 ====>  100.00% 완료  1490.7667


all 

### 02. 예측 결과

In [2]:
result_df = result_df[['어제 날짜', '종목', '어제 종가', '오늘 고가', '오늘 고가 변동폭 예측']]

In [3]:
result_df['오늘 고가 변동폭 예측'] = result_df['오늘 고가 변동폭 예측'].apply(lambda x : round(x,2))

In [4]:
result_df

,어제 날짜,종목,어제 종가,오늘 고가,오늘 고가 변동폭 예측
0,20200318,060310,1705.0,1944.0,14.02
1,20200318,095570,2895.0,2946.0,1.76
2,20200318,068400,6840.0,7605.0,11.18
3,20200318,006840,15450.0,14561.0,-5.75
4,20200318,054620,5980.0,6187.0,3.46
...,...,...,...,...,...
2139,20200318,189980,986.0,1082.0,9.74
2140,20200318,000540,1730.0,1888.0,9.13
2141,20200318,003280,229.0,249.0,8.73
2142,20200318,037440,3700.0,3830.0,3.51


### 03. 내일 고가 변동폭이 가장 높은 TOP 10

In [5]:
result_df = result_df.sort_values(by='오늘 고가 변동폭 예측', ascending=False).set_index('종목')

In [6]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] < 30.00]

In [7]:
result_df[:10]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
308100,20200318,6980.0,9061.0,29.81
195500,20200318,5220.0,6713.0,28.60
033320,20200318,2510.0,3212.0,27.97
009270,20200318,1145.0,1460.0,27.51
217620,20200318,1005.0,1281.0,27.46
019540,20200318,1890.0,2404.0,27.20
196490,20200318,880.0,1115.0,26.70
005950,20200318,6020.0,7622.0,26.61
222980,20200318,3650.0,4617.0,26.49


### 04. 내일 고가 변동폭이 가장 높은 TOP 11 ~ 20

In [8]:
result_df[11:20]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
004270,20200318,1515.0,1911.0,26.14
021080,20200318,1205.0,1519.0,26.06
078070,20200318,17100.0,21511.0,25.80
214330,20200318,2450.0,3082.0,25.80
064480,20200318,1695.0,2123.0,25.25
900300,20200318,781.0,975.0,24.84
197140,20200318,3050.0,3805.0,24.75
011280,20200318,2725.0,3398.0,24.70
001550,20200318,8800.0,10924.0,24.14


### 05. 내일 고가 변동폭이 가장 낮은 BOTTOM 10

In [9]:
result_df = result_df[result_df['오늘 고가 변동폭 예측'] > -30.00]
result_df[-10:]

,어제 날짜,어제 종가,오늘 고가,오늘 고가 변동폭 예측
종목,,,,
154040,20200318,1520.0,1166.0,-23.29
190510,20200318,10950.0,8345.0,-23.79
900100,20200318,717.0,543.0,-24.27
234920,20200318,2245.0,1675.0,-25.39
138690,20200318,6160.0,4584.0,-25.58
275630,20200318,3460.0,2574.0,-25.61
257370,20200318,8600.0,6335.0,-26.34
299910,20200318,5970.0,4389.0,-26.48
014280,20200318,2185.0,1601.0,-26.73


### 05. 특정 종목 검색

In [10]:
# result_df.loc['019770']